In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from game.hyperparams import Hp

import pickle
import glob
import os


import glob
import os
import pickle
import sys
import inspect

sys.path.append(os.getcwd())
sys.path.append("")
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

import torch

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import numpy as np
from pandas import DataFrame


from game.hint_play_game import TwoRoundHintGame
from train_qlearner import obs_to_agent

from game.hyperparams import Hp
from agent.qlearner import QLearner

GPU not available, CPU used


In [2]:
hp_default = Hp(hand_size=5,
                          nlab1=3,
                          nlab2=3,
                          )

In [5]:
agent1s = []
agent2s = []
agent_paths = sorted(glob.glob(os.path.join("res/Att3_hs_5_l1_3_l2_3_FalseFalse4000000", "*.pkl")))
cluster1 = [agent_paths[i] for i in [4,6,10,12,13,17]]
cluster2 = [agent_paths[i] for i in  [1,2,3,5,7,8,9,11,14,15,16,]]

for filename in cluster1:
       with open(filename, "rb") as f:
               res = pickle.load(f)
               agent1s += [res['p1']]
               agent2s += [res['p2']]

In [44]:
# hand1 = [np.array([0., 1., 0., 1., 0., 0.]),
#          np.array([0., 1., 0., 0., 1., 0.]),
#          np.array([1., 0., 0., 0., 0., 1.]),
#          np.array([1., 0., 0., 0., 0., 1.]),
#          np.array([1., 0., 0., 0., 0., 1.]),]

playable_card_num=4

hand1 = [np.array([0., 1., 0., 1., 0., 0.]),
         np.array([0., 1., 0., 0., 1., 0.]),
         np.array([0., 1., 0., 0., 1., 0.]),
         np.array([0., 1., 0., 0., 1., 0.]),
         np.array([0., 1., 0., 0., 1., 0.]),]
         
hand2 = [np.array([0., 1., 0., 0., 1., 0.]),
         np.array([0., 0., 1., 1., 0., 0.]),
         np.array([0., 0., 1., 1., 0., 0.]),
         np.array([0., 0., 1., 1., 0., 0.]),
         np.array([0., 0., 1., 1., 0., 0.]),]


def get_given_initial_state(hp=hp_default, hand1=hand1, hand2=hand2, playable_card_num=4):
    """
    Generate initial obs assignment
    P1 gets all hands as well as the playable card for P2
    P2 gets all hands as well as an empty card
    """
    info = {}
    o1=hand1 
    o2=hand2
    o1.append(o2[playable_card_num])
    o2.append(np.zeros((hp.nlab1 + hp.nlab2)))
    o1 = np.array(o1).flatten()
    o2 = np.array(o2).flatten()
    info['playbale_card_num'] = playable_card_num
    return (o1, o2), info

In [45]:
obs, info = get_given_initial_state()
initial_config = {'obs': obs, 'info':info}

In [46]:
def sample_games(p1, p2, episodes=10, verbose=False, initial_config=initial_config):
    hp1 = p1.hp
    hp2 = p2.hp
    env = TwoRoundHintGame(hp=hp1, initial_config=initial_config)

    hints = []
    plays = []

    print(f'agents are {hp1.agent_type} and {hp2.agent_type}')

    for i_episode in range(episodes):
        # Initialize the environment and state
        obs1, info = env.reset()
        if verbose:
            env.render()
        obs1_a1, obs1_a2 = obs_to_agent(obs1, hp=hp1)
        if hp1.agent_type != 'FF':
            obs1_a1 = obs1_a1.reshape(-1, hp1.nlab1 + hp1.nlab2).T
        # P1 select and perform a hint
        # obs1_a1 = torch.tensor(obs1_a1, device=device)
        a1 = p1.select_action(torch.tensor([obs1_a1], device=device), evaluate=True)
        obs2, _, _, _ = env.step(a1)
        hint_card = env.get_card_symbol_in_round()
        if verbose:
            env.render()
        # P2 plays a card
        obs2_a1, obs2_a2 = obs_to_agent(obs2, hp=hp2)
        if hp2.agent_type != 'FF':
            obs2_a2 = obs2_a2.reshape(-1, hp2.nlab1 + hp2.nlab2).T
        a2 = p2.select_action(torch.tensor([obs2_a2], device=device), evaluate=True)
        _, r, _, _ = env.step(a2)
        r = torch.tensor([r[0]], device=device)
        play_card = env.get_card_symbol_in_round()
        if verbose:
            env.render()
        hints.append(hint_card)
        plays.append(play_card)
    return hints, plays

In [56]:
hint_sym = []
play_sym = []

for i in range(len(agent1s)):
    p1 = agent1s[i]
    p2 = agent2s[i]
    hints, plays = sample_games(p1, p2, verbose=False, episodes=100)
    hint_sym += hints
    play_sym += plays
    print(Counter(hints))


agents are Att3 and Att3
Counter({'2A': 84, '2B': 16})
agents are Att3 and Att3
Counter({'2B': 100})
agents are Att3 and Att3
Counter({'2A': 94, '2B': 6})
agents are Att3 and Att3
Counter({'2A': 100})
agents are Att3 and Att3
Counter({'2B': 100})
agents are Att3 and Att3
Counter({'2A': 100})


In [57]:
from collections import Counter

In [58]:
Counter(hint_sym)

Counter({'2B': 222, '2A': 378})

In [59]:
Counter(play_sym)

Counter({'3A': 598, '2B': 2})

In [53]:
hint_sym = []
play_sym = []

for i in range(len(agent1s)):
    for j in range(len(agent1s)):
        if i != j:
            p1 = agent1s[i]
            p2 = agent2s[j]
            hints, plays = sample_games(p1, p2, verbose=False, episodes=100)
            hint_sym += hints
            play_sym += plays
            print(i,j,Counter(hints), Counter(plays))

agents are Att3 and Att3
0 1 Counter({'2A': 95, '2B': 5}) Counter({'3A': 99, '2B': 1})
agents are Att3 and Att3
0 2 Counter({'2A': 96, '2B': 4}) Counter({'3A': 100})
agents are Att3 and Att3
0 3 Counter({'2A': 90, '2B': 10}) Counter({'3A': 100})
agents are Att3 and Att3
0 4 Counter({'2A': 85, '2B': 15}) Counter({'3A': 100})
agents are Att3 and Att3
0 5 Counter({'2A': 88, '2B': 12}) Counter({'3A': 88, '2B': 12})
agents are Att3 and Att3
1 0 Counter({'2B': 99, '2A': 1}) Counter({'3A': 90, '2B': 10})
agents are Att3 and Att3
1 2 Counter({'2B': 100}) Counter({'3A': 100})
agents are Att3 and Att3
1 3 Counter({'2B': 100}) Counter({'3A': 100})
agents are Att3 and Att3
1 4 Counter({'2B': 100}) Counter({'3A': 100})
agents are Att3 and Att3
1 5 Counter({'2B': 99, '2A': 1}) Counter({'2B': 99, '3A': 1})
agents are Att3 and Att3
2 0 Counter({'2A': 84, '2B': 16}) Counter({'3A': 100})
agents are Att3 and Att3
2 1 Counter({'2A': 82, '2B': 18}) Counter({'3A': 100})
agents are Att3 and Att3
2 3 Counter(

In [54]:
Counter(hint_sym)

Counter({'2A': 1869, '2B': 1131})

In [55]:
Counter(play_sym)

Counter({'3A': 2746, '2B': 254})

In [28]:
from sample.sample import sample_games

game_num = 2

In [37]:
i=5
p1 = agent1s[i]
p2 = agent2s[i]
sample_games(p1, p2, episodes=game_num, verbose=True)

===== Game starts with handsize: 5, nlabl: 3, nlab2: 3 =====
Playable hand is: 1C
Agent 1 hand is: ['3B', '3A', '1̲C', '1̲C', '2B']
Agent 2 hand is: ['2B', '3C', '2B', '1̲C', '1A']
Agent 1 hints 1C @  3
Agent 2 plays 1C @  3
Final reward of game: 1.0


===== Game starts with handsize: 5, nlabl: 3, nlab2: 3 =====
Playable hand is: 3C
Agent 1 hand is: ['1B', '1A', '3B', '2B', '1C']
Agent 2 hand is: ['3B', '2B', '3̲C', '3B', '2B']
Agent 1 hints 1C @  4
Agent 2 plays 3C @  2
Final reward of game: 1.0




array([1., 1.], dtype=float32)